# Практика: Prompt Engineering

## Введение
В данном задании мы будем работать с API онлайн моделей через together.ai. Эти модели предоставляют $5 кредита при регистрации, что позволит вам провести необходимые эксперименты. Вначале мы познакомимся с API на практике, а затем выполним три основных задания.

---

## Задача 1: Знакомство с API together.ai
1. Зарегистрируйтесь на платформе [together.ai](https://together.ai/) и получите API ключ.
2. Используйте приведенный ниже код для вызова модели Llama через together.ai:


In [ ]:
import requests
import json

In [ ]:
# Вставьте свой API ключ
API_KEY = "Ваш ключ из https://api.together.ai/"
# Не забудьте удалить ключ перед сдачей задания

# Параметры модели
url = "https://api.together.ai/v1/completions"
data = {
    "model": "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    "prompt": "Translate the following English text to French: 'Hello, how are you?'",
    "max_tokens": 50
}
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

response = requests.post(url, headers=headers, json=data)
if response.status_code == 200:
    print("Response:", response.json()["choices"][0]["text"])
else:
    print("Error:", response.status_code, response.text)

Выше описан пример запроса в completion формате, то есть подается поле `prompt`, которое напрямую подается в модель. Как мы помним, у Llama 3.x моделей есть свой формат входных данных, так что лучше подавать его. Отформатируем наш запрос.

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("unsloth/Meta-Llama-3.1-8B-Instruct")
prompt = tokenizer.apply_chat_template(
    [{"role": "user", "content": "Translate the following English text to French: 'Hello, how are you?'"}],
    add_generation_prompt=True,
    tokenize=False
)
print(prompt)

И пошлем его в API:

In [ ]:
data = {
    "model": "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    "prompt": prompt,
    "max_tokens": 50
}

response = requests.post(url, headers=headers, json=data)
if response.status_code == 200:
    print("Response:", response.json()["choices"][0]["text"])
else:
    print("Error:", response.status_code, response.text)

Это еще не все! Чтобы не заниматься форматированием на стороне клиента, почти все провайдеры поддерживают работу с сообщениями и ролями и берут работу по форматированию на себя. Для этого вместо поля prompt нужно послать поле messages.

In [ ]:
data = {
    "model": "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    "messages": [{"role": "user", "content": "Translate the following English text to French: 'Hello, how are you?'"}],
    "max_tokens": 50
}

response = requests.post(url, headers=headers, json=data)
if response.status_code == 200:
    print("Response:", response.json()["choices"][0]["text"])
else:
    print("Error:", response.status_code, response.text)

3. Модифицируйте запрос, чтобы:
   - Решить простую математическую задачу (например, сложение чисел).
   - Сгенерировать текст на тему "Как искусственный интеллект меняет мир".


## Задача 2: Решение математических задач через Chain of Thought

Используя подход Chain of Thought (CoT), решите 10 математических задач и измерьте accuracy модели.


1. Создайте функцию, которая формирует запросы для модели с использованием CoT:

In [ ]:
def solve_math_cot(prompt: str) -> str:
    cot_prompt = f"Давайте подумаем шаг за шагом, чтобы решить эту задачу: {prompt}"
    # Подставьте сюда вызов API
    return response_text

2. Подготовьте 5 задач (например, из школьной программы) и выполните их решение через модель.

In [ ]:
# Пример запроса для задачи умножения
example_prompt = "Чему равно 23 умножить на 47?"
cot_prompt = f"Давайте подумаем шаг за шагом, чтобы решить эту задачу: {example_prompt}"

data = {
    "model": "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    "prompt": cot_prompt,
    "max_tokens": 100
}

response = requests.post(url, headers=headers, json=data)
if response.status_code == 200:
    print("Ответ:", json.loads(response.text)["choices"][0]["text"].strip())
else:
    print("Ошибка:", response.status_code, response.text)


3. Подсчитайте количество правильно решённых задач (accuracy).

In [ ]:
My_accuracy =

## Задача 3: Классификация IMDB через few-shot и zero-shot

Проведите классификацию отзывов IMDB на позитивные и негативные с использованием few-shot и zero-shot подходов.


1. Выберите 5 примеров для few-shot обучения (например, 2 позитивных и 3 негативных отзыва).
2. Реализуйте запросы к модели в режиме zero-shot и few-shot:

In [ ]:
def classify_review(prompt: str, examples: List[str] = None) -> str:
    few_shot_prompt = """
    {examples}
    Теперь классифицируйте отзыв: {prompt}
    """ if examples else f"Классифицируйте следующий отзыв: {prompt}"
    # Подставьте сюда вызов API
    return response_text

In [ ]:
# Пример zero-shot классификации
review_prompt = "Этот фильм был потрясающим! Сюжет увлекательный, а актеры великолепны."
zero_shot_prompt = f"Классифицируйте следующий отзыв как позитивный или негативный: {review_prompt}"

data = {
    "model": "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    "prompt": zero_shot_prompt,
    "max_tokens": 50
}

response = requests.post(url, headers=headers, json=data)
if response.status_code == 200:
    print("Классификация:", json.loads(response.text)["choices"][0]["text"].strip())
else:
    print("Ошибка:", response.status_code, response.text)


3. Сравните результаты, объяснив различия между zero-shot и few-shot подходами.

## Задача 4: Self-reflection и качество ответов модели

Проверьте, как self-reflection влияет на качество ответов модели.


1. Реализуйте функцию self-reflection, которая анализирует ответ модели и предлагает улучшения:

In [ ]:
def self_reflection(prompt: str) -> str:
    reflection_prompt = f"Проанализируйте ответ и предложите улучшения: {prompt}"
    # Подставьте сюда вызов API
    return response_text

2. Используйте self-reflection для 5 задач из задачи 2 (CoT) и сравните результаты до и после рефлексии.
3. Ответьте на вопросы:
   - Улучшаются ли ответы?
   - Исправляет ли модель правильные ответы на неправильные?

## Задача 5: Защита от инъекций

 Исследуйте методы защиты от инъекций в пользовательских вводах.


1. Реализуйте функцию, которая проверяет ввод пользователя на наличие потенциальных инъекций:

In [ ]:
import re

# Функция проверки на инъекцию
def detect_injection(user_input: str) -> bool:
    """
    Проверяет текст на наличие возможных инъекций.
    Возвращает True, если найдена инъекция.
    """
    # Примеры подозрительных шаблонов
    injection_patterns = [
        r"ignore.*instructions",  # Игнорировать инструкции
        r"forget.*previous",      # Забыть предыдущие команды
        r"reveal.*secret",        # Раскрыть секрет
        r"break.*rules",          # Нарушить правила
    ]
    for pattern in injection_patterns:
        if re.search(pattern, user_input, re.IGNORECASE):
            return ...
    return False

# Пример использования
def process_user_input(user_input: str) -> str:
    """
    Обрабатывает пользовательский ввод с проверкой на инъекции.
    """
    if detect_injection(user_input):
        return "Ошибка: обнаружена потенциальная инъекция!"

    # Если инъекций нет, отправляем запрос к модели
    data = {
        "model": "meta-llama/Llama-3.3-70B-Instruct-Turbo",
        "prompt": user_input,
        "max_tokens": 50
    }
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return json.loads(response.text)["choices"][0]["text"].strip()
    else:
        return f"Ошибка: {response.status_code}"



2. Протестируйте функцию на 5 различных вводах, включая как корректные запросы, так и попытки инъекций.
3. Напишите выводы о том, как система справляется с защитой и какие улучшения можно внести.

In [ ]:
# Тестирование
inputs = [
    "What is the capital of France?",
    ...
]

for i, inp in enumerate(inputs):
    print(f"Input {i+1}: {inp}")
    print(f"Output: {process_user_input(inp)}")
    print("-" * 40)

## Требования к оформлению
- Каждый результат должен быть сопровожден кодом, комментариями и выводами.
- Предоставьте accuracy, сравнения и выводы в формате markdown в jupyter notebook.

## Дополнительное задание (по желанию)
Проверьте, как работает модель с разными длинами промпта (от коротких до детализированных). Как длина промпта влияет на качество ответа?

---

